# Master File

In [6]:
from pa_lib.data import desc_col

## Load data

In [2]:
# make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path

file_dir = Path.cwd()
parent_dir = file_dir.parent
sys.path.append(str(parent_dir))

In [8]:
from vkprog_data_prep import bd_train_scoring

(training_all, scoring_all, feature_colnames_bd, feature_colnames_dates) = bd_train_scoring(
    day=23,
    month=9,
    year_score=2019,
    year_train=2018,
    year_span=4,
    scale_features=True)

2019-09-23 12:50:34 [INFO] Reading from file C:\Users\stc\data\vkprog\bd_data.feather
2019-09-23 12:50:35 [INFO] Finished loading binary file in 0.16s (0.75s CPU)
2019-09-23 12:50:36 [INFO] Period: KW_2
2019-09-23 12:50:36 [INFO] Calculate Reservation...
2019-09-23 12:50:36 [INFO] Calculate Aushang...
2019-09-23 12:50:36 [INFO] Merge Results...
2019-09-23 12:50:37 [INFO] aggregate_bookings: Done.
2019-09-23 12:50:37 [INFO] Computing: Pivot Table
2019-09-23 12:50:37 [INFO] Running: booking_yearly_totals(YYYYKW, year_span) 
2019-09-23 12:50:37 [INFO] Computing: Yearly total sums
2019-09-23 12:50:37 [INFO] Merging: Left-Join to Container dataframe
2019-09-23 12:50:37 [INFO] Merging: Left-Join to Container dataframe
2019-09-23 12:50:37 [INFO] Merging: Left-Join to Container dataframe
2019-09-23 12:50:37 [INFO] Merging: Left-Join to Container dataframe
2019-09-23 12:50:37 [INFO] Final merge
2019-09-23 12:50:37 [INFO] Computing: Pivot Table
2019-09-23 12:50:38 [INFO] Running: booking_yearly_

In [9]:
training_all.describe()

,Endkunde_NR,Erste_Buchung_Delta,Letzte_Buchung_Delta,Erste_Letzte_Buchung_Delta,Netto_Sum_Aus_RY_0_KW_1,Netto_Sum_Aus_RY_0_KW_11,Netto_Sum_Aus_RY_0_KW_13,Netto_Sum_Aus_RY_0_KW_15,Netto_Sum_Aus_RY_0_KW_17,Netto_Sum_Aus_RY_0_KW_19,...,Target_Sum_Aus_RY_0_KW_23,Target_Sum_Res_RY_0_KW_23,Netto_Sum_Aus_RY_0,Netto_Sum_Res_RY_0,Netto_Sum_Aus_RY_1,Netto_Sum_Res_RY_1,Netto_Sum_Aus_RY_2,Netto_Sum_Res_RY_2,Netto_Sum_Aus_RY_3,Netto_Sum_Res_RY_3
count,18163.000000,18163.000000,18163.000000,18163.000000,18163.000000,18163.000000,18163.000000,18163.000000,18163.000000,18163.000000,...,18163.000000,18163.000000,18163.000000,18163.000000,18163.000000,18163.000000,18163.000000,18163.000000,18163.000000,18163.000000
mean,483311.851236,0.402401,0.241557,0.292603,0.032521,0.018302,0.028564,0.016953,0.029988,0.017117,...,0.014833,0.014902,0.265689,0.262621,0.274739,0.265098,0.279331,0.265944,0.265493,0.270983
std,186062.690442,0.284042,0.260728,0.305737,0.142893,0.106569,0.128751,0.101891,0.135093,0.103942,...,0.095103,0.095220,0.281295,0.285366,0.284024,0.281086,0.283268,0.276950,0.289992,0.292102
min,100034.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,472542.500000,0.181818,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,572615.000000,0.363636,0.200000,0.181818,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.359393,0.000000,0.000000,0.000000
75%,612353.000000,0.727273,0.400000,0.545455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.534813,0.541199,0.541228,0.533121,0.541459,0.526768,0.546833,0.551898
max,651482.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
#desc_col(training_all)

In [ ]:
temp_df = training_all
print("Target_Res_flg == False")
boxplot_histogram(temp_df.loc[ temp_df.Target_Res_flg == False ,"Erste_Buchung_Delta"])
print("Target_Res_flg == True")
boxplot_histogram(temp_df.loc[ temp_df.Target_Res_flg == True ,"Erste_Buchung_Delta"])

print("Target_Res_flg == False")
boxplot_histogram(temp_df.loc[ temp_df.Target_Res_flg == False ,"Letzte_Buchung_Delta"])
print("Target_Res_flg == True")
boxplot_histogram(temp_df.loc[ temp_df.Target_Res_flg == True ,"Letzte_Buchung_Delta"])


print("Target_Res_flg == False")
boxplot_histogram(temp_df.loc[ temp_df.Target_Res_flg == False ,"Erste_Letzte_Buchung_Delta"])
print("Target_Res_flg == True")
boxplot_histogram(temp_df.loc[ temp_df.Target_Res_flg == True ,"Erste_Letzte_Buchung_Delta"])